# Exercise: Cascading Processes on Networks


In this exercise, we implement and analyse the models presented in Lecture.  We provide you with an almost complete mesa model that implements all the model variants discussed in the lecture.
Only the simplest *constant load inward and outward variants* are left out for you to implement. 
Then, with the model completed, you can analyse it in an interactive Jupyter Notebook.
This is a good general approach: writing the highly structured and reusable core code in python modules (or packages for more complex projects) and then calling that code in a Notebook in which you perform the analysis and generate results/reports.


### Tasks
1. **Complete the `inward` and `outward` model variants**  
    Inspect the code provided to you.
    You find incomplete code stubs starting on line 75 in `agent.py`, which you should complete.
    
2. **Test your implementation**  
    Run the model variant you implemented for known cascade processes and confirm the correctness of your implementation.
    
3. **Compute the $(\mu, \sigma)$ phase diagram seen in the lecture**  
    In the lecture you saw the results of calculations from the macroscopic perspective to the problem for varying means and variances of the load distribution.  Here, you are asked to reproduce the  $(\mu, \sigma)$ diagram through microscopic ABM simulation


In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from model import CascadeNetwork

## Task 2: run the model

Once you finish Task 1 (see above), you can test if the model works correctly.

The adjacency matrix, loads and capacities shown here are taken from the original paper by Lorenz et. al., 
`DOI: 10.1140/epjb/e2009-00347-4`

You can find the rest of the examples in the paper implemented in `test.py`.  
Note that you can run that file in command line and see the model evolution reported over time:
```
$ python3 test.py
Step 0:
Node 0 (Capacity = 1.7): Load 1.00, non-failed
Node 1 (Capacity = 1.7): Load 1.00, non-failed
Node 2 (Capacity = 1.3): Load 1.00, non-failed
Node 3 (Capacity = 1.3): Load 1.00, non-failed
...
```

In [ ]:
adj_matrix = np.matrix([[0, 0, 1, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 0, 0, 0, 0],
                        [1, 1, 0, 1, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 1, 1, 1, 1],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0, 0]])

load_vec_7 = [1, 1, 3.6, 1, 1, 1, 1, 1, 1]
capacity_vec_7 = [1.7, 1.7, 1.3, 1.3, 1.5, 1.55, 1.55, 1.55, 1.55]

In [ ]:
# instantiate model and run it with model_type="constant", load_type="in" and test=True
# YOUR CODE HERE

#### Testing your implementation

Note, that it is difficult to judge if your implementation is correct from the above example.
Instead, try the model for a smaller network, for which first compute the cascade by hand. Then, compare the simulation and the result computed by hand.

In [ ]:
## THIS TASK IS TECHNICALLY STRAIGHFORWARD TO DO, 
## AS YOU CAN JUST ADJUST THE VALUES IN THE ABOVE TWO CELLS
##
## INSTEAD, LET US HAVE SOME FUN HERE AND SEE IF WE CAN BUILD THE ADJACENCY MATRIX DIFFERENTLY

# Define how many nodes we want to have in the network
nnodes = 5
# Define the network by means of an "edge list"
edges = [
    (0,1),
    (0,2),
    (0,3),
    (3,4),
]

## Now, let us build the adjacency matrix from the edge list
## First, we create an 2 dimensional array of the correct size, populated with zeros (no edges)
adj_test = np.zeros((nnodes, nnodes))

## Next, we need to set to ones the elements that correspond to edges in the network
## A clean and readable solution is to write a for-loop:
for i,j in edges:
    adj_test[i,j] = 1

## Pay attention to the fact that we used tuple "unpacking"
## I.e., each step in the for loop is one edge, but one edge is a tuple of two elements
## So we assigned the variables i and j to the elements of this tuple in the same line as the for loop declaration


## Alternatively, let as do something less readable, but what showes some new functionality
adj_test1 = np.zeros((nnodes, nnodes))
adj_test1[tuple(zip(*edges))] = 1

print('Are the two matrices the same?')
print(adj_test == adj_test1)
## what we did above is to reshape `edges` into two tuples, ((0, 0, 0, 3), (1, 2, 3, 4)),
## which is a valid way to index multi-dimensional numpy arrays
## ( https://docs.scipy.org/doc/numpy-1.13.0/user/basics.indexing.html#indexing-multi-dimensional-arrays )
## Here, pay attention to the very useful function `zip`, which does what the name says:
## it "zips" iterables elementwise, like the two zipper sides do on your jacket
## Interestingly, zip function is its own inverse function,
## i.e., it "unzips" a `zippedobject` if you do zip(*zippedobject)


## Now, let us assume our network is not directed
## To implement that, we need to make the adjacency matrix symmetric
## Given that we have each edge in the edge list only ones (as (i,j) or (j,i), but not both)
# We can get our undirected adjacency matrix by adding the transpose of the directed one to itself
adj_test += adj_test.T

print("\nOur adjacency matrix")
print(adj_test)

Now, let us run the *inward* and *outward* variants of the model for the given adjacency and the following loads and capacities.

You can confirm the correctness of the model dynamics by simple manual calculation based on formulas in Slides 15 and 20 of the lecture

In [ ]:
load_vec_test = [0,0,0,0,0]
capacity_vec_test = [0, .7, .3, .8, .5]

In [ ]:
# instantiate model and run it with model_type="constant", load_type="in" and test=True
# YOUR CODE HERE

In [ ]:
# instantiate model and run it with model_type="constant", load_type="out" and test=True
# YOUR CODE HERE

## Task 3: Phase diagram

In [ ]:
def fraction_failed(model):
    """Return the fraction of the failed nodes for the given `model`"""
    states = [node.failed for node in model.schedule.agents]
    N = model.num_nodes
    return sum(states) / N

In [ ]:
fixed_params = {'adj_matrix': adj_matrix,
                'load_vec': np.ones(adj_matrix.shape[0]), # all load equal to one
                'model_type': 'constant', 
                'load_type': 'in'}


mus = np.arange(0, 1, step=0.02)
sigmas = np.arange(0, 1, step=0.02)
repetitions = 5

In [ ]:
average_cascade_size = np.zeros((len(mus), len(sigmas)))

for i, mu in enumerate(mus):
    for j, sigma in enumerate(sigmas):
        sizes = []
        for _ in range(repetitions):
            # Generate capacity vector 'c_vec' using 'np.random.normal'
            c_vec = # YOUR CODE HERE
            # Initialize model with 'c_vec' and 'fixed_params'
            model = # YOUR CODE HERE
            
            while model.running and model.schedule.steps < 100:
                model.schedule.steps += 1  # Simulate model step
                model.step()
            
            sizes.append(fraction_failed(model))
        
        average_cascade_size[i, j] = np.mean(sizes)

Now, let us plot the phase diagram.

Display the correct values of $\mu$ and $\sigma$ on the axes. Refer to `pyplot.imshow` documentation to learn how.

In [ ]:
# YOU PLOTTING CODE HERE